In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

#pull data
unsorted_data = pd.DataFrame.from_csv("/Users/Mahir/Documents/Personal/Comp Sci/analyzing_lesions/clinical_info.csv").drop('mse',axis=1)
#unsorted_data = pd.DataFrame.from_csv("/data/henry1/mahamber/clinical_info.csv").drop('mse',axis=1)

#gather only SP and RR from available cohort
rr = unsorted_data['dc'] == 'RR'
sp = unsorted_data['dc'] == 'SP'
rr_df = unsorted_data[rr]
sp_df = unsorted_data[sp]
scores_lesion = rr_df.append(sp_df)
scores_lesion.columns

## ROC and Permutation Test

In [ ]:
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedShuffleSplit, permutation_test_score

#set X and y values

#X values are (309,3) array, accounting for total juxtacortical lesions, total lesions, and EDSS scores as features
X_values=scores_lesion[['jux_total_lesions','total_lesions','edss']].values
#y values are (309,) array, accounting for one of two disease courses, which is then binarized for ROC analysis
y_values=label_binarize(scores_lesion.dc.values, classes=['RR','SP']).ravel()

#set classifier and cross validation technique
cv = StratifiedShuffleSplit(y_values,n_iter=10,test_size=0.33,random_state=0)
clf = svm.SVC(kernel="linear", C=1, probability=True) #consider including weights for each feature?

#calculate permutations
score, perm_score, pvalue = permutation_test_score(clf,
                                                   X_values,
                                                   y_values,
                                                   scoring="accuracy",
                                                   cv=cv,
                                                   n_permutations=100,
                                                   n_jobs=1)
print("Classification score: %2.3f (p value: %2.5f)" % (score,pvalue))

#plot permutations test
plt.figure(figsize=(10,6))
plt.hist(perm_score,20,label="Permutation Scores")
ylim=plt.ylim()
plt.plot([score,score],ylim,'--g',linewidth=3,label = 'Classification Score (pvalue: %2.5f)' %pvalue)
plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.title('Permutation Test - 50 Permutations',size=16)

#calculate and plot ROC curves for each ShuffledSplit
plt.figure(figsize=(9,9))
for i, (train_indexes, test_indexes) in zip(range(len(y_values)), cv): 
    X_train = X_values[train_indexes] 
    y_train = y_values[train_indexes] 
    X_test = X_values[test_indexes] 
    y_test = y_values[test_indexes] 
    clf = clf.fit(X_train, y_train) 
    prob = clf.predict_proba(X_test)
    fpr, tpr, thres = roc_curve(y_test,prob[:,1])
    auc_data = auc(fpr,tpr)
    plt.plot(fpr,
             tpr,
             lw=1,
             label="ROC Curve Fold %d, Area = %0.2f" % (i, auc_data))
plt.plot([0,1],[0,1],
         lw=5,
         label="ROC Curve, Chance",
         color='gray')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])
plt.xlabel('False Positive Rate',size=12)
plt.ylabel('True Positive Rate',size=12)
plt.title('ROC Analysis', size=16)
plt.legend(loc='lower right')

#show both plots
plt.tight_layout()
plt.show()

In [ ]:
#extra trees classifier / feature importances
from sklearn import svm
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedShuffleSplit, permutation_test_score
from sklearn.ensemble import ExtraTreesClassifier

#set X and y values

#X values are (309,3) array, accounting for total juxtacortical lesions, total lesions, and EDSS scores as features
X_values=scores_lesion.drop(['dc','dd','edss','msfc 25 1','msfc 25 2'],axis=1).values
X_labels=scores_lesion.drop(['dc','dd','edss','msfc 25 1','msfc 25 2'],axis=1).columns.values
#y values are (309,) array, accounting for one of two disease courses, which is then binarized for ROC analysis
y_values=label_binarize(scores_lesion.dc.values, classes=['RR','SP']).ravel()

#feature rankings
forest = ExtraTreesClassifier(n_estimators=250,random_state=0)
cv = StratifiedShuffleSplit(y_values,n_iter=10,test_size=0.33,random_state=0)
for i, (train_indexes, test_indexes) in zip(range(len(y_values)), cv): 
    X_train = X_values[train_indexes] 
    y_train = y_values[train_indexes] 
    X_test = X_values[test_indexes] 
    y_test = y_values[test_indexes] 
    importances = forest.fit(X_train,y_train).feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],axis=0)
    indices = np.argsort(importances)[::-1]
    print("Feature ranking (fold %s):" % i)
    for f in range(3):
        print("%d. feature %d (%s) (%f)" % (f + 1, indices[f], X_labels[indices[f]], importances[indices[f]]))
    plt.figure()
    plt.title("Feature importances (fold %s)" % i)
    plt.bar(range(X_values.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X_values.shape[1]), X_labels[indices],rotation=90)
    plt.xlim([-1, X_values.shape[1]])
    
plt.tight_layout()
plt.show()

In [ ]:
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedShuffleSplit, permutation_test_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.learning_curve import learning_curve

start = time.time()

#set X and y values

#X values are (309,3) array, accounting for total juxtacortical lesions, total lesions, and EDSS scores as features
X_values=scores_lesion.drop(['dc','dd','edss','msfc 25 1','msfc 25 2'],axis=1).values
#y values are (309,) array, accounting for one of two disease courses, which is then binarized for ROC analysis
y_values=scores_lesion.edss.values
current = time.time(); elapsed = current-start; print "Values done, time elapsed:",elapsed; print
print y_values
#set classifier and cross validation technique
cv = StratifiedShuffleSplit(y_values,n_iter=10,test_size=0.33,random_state=0)
clf = ExtraTreesRegressor(n_estimators=500,random_state=0) #consider including weights for each feature?
current = time.time(); elapsed = current-start; print "Classifers done, time elapsed:",elapsed; print
#calculate and plot ROC curves for each ShuffledSplit
plt.figure(figsize=(9,9))
train_sizes, train_scores, test_scores = learning_curve(clf, X_values, y_values, cv=cv, n_jobs=1, train_sizes=np.linspace(0.1, 1.0, 5))
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
current = time.time(); elapsed = current-start; print "Calculations done, time elapsed:",elapsed; print

plt.xlabel("Training examples")
plt.ylabel("Score")    
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

plt.legend(loc="best")
#show both plots
plt.tight_layout()
plt.show()
current = time.time(); elapsed = current-start; print "Total time elapsed:",elapsed; print

## Code Created during Learning Process

In [ ]:
#basic cv test
from sklearn import svm
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn import metrics

#set X and y values
X_values=scores_lesion[['inf_total_lesions','inf_total_volume_of_lesions']]
y_values=scores_lesion.dc

X_train,X_test,y_train,y_test = cross_validation.train_test_split(X_values,y_values,test_size=0.50,random_state=0)

#set classifier
clf = svm.SVC(kernel='poly', C = 1,degree=2).fit(X_train,y_train)
prediction = clf.predict(X_test)

#determine number of correct and incorrect
number_correct = 0
number_wrong = 0
for count in range(len(y_test)):
    if prediction[count] == y_test.values[count]:
        number_correct += 1
    else:
        number_wrong += 1
print "Correct: {}  Incorrect: {}".format(number_correct,number_wrong)

In [ ]:
#basic scatterplot
print len(y_values)

x = scores_lesion.edss
y = scores_lesion.inf_total_volume_of_lesions
fit = np.polyfit(x,y,1)
fit_fn=np.poly1d(fit)

plt.figure(figsize=(10,8))
plt.plot(x,y,'yo',x,fit_fn(x),'--k')
plt.xlabel('EDSS')
plt.xlim(-1,)
plt.ylim(-1,)
plt.show()

In [ ]:
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import StratifiedKFold, permutation_test_score

#set X and y values

#X values are (309,3) array, accounting for total juxtacortical lesions, total lesions, and EDSS scores as features
X_values=scores_lesion[['jux_total_lesions','inf_total_lesions']].values
#y values are (309,) array, accounting for one of two disease courses, which is then binarized for ROC analysis
y_values=label_binarize(scores_lesion.dc.values, classes=['RR','SP']).ravel()

#set classifier and cross validation technique
cv = StratifiedKFold(y_values, n_folds=10)
clf = svm.SVC(kernel="linear", C=1, probability=True)

#Look into random forest

#calculate permutations
score, perm_score, pvalue = permutation_test_score(clf,
                                                   X_values,
                                                   y_values,
                                                   scoring="accuracy",
                                                   cv=cv,
                                                   n_permutations=50,
                                                   n_jobs=1)
print("Classification score: %2.3f (p value: %2.5f)" % (score,pvalue))

#plot permutations test
plt.figure(figsize=(10,6))
plt.hist(perm_score,20,label="Permutation Scores")
ylim=plt.ylim()
plt.plot([score,score],ylim,'--g',linewidth=3,label = 'Classification Score (pvalue: %2.5f)' %pvalue)
plt.ylim(ylim)
plt.legend()
plt.xlabel('Score')
plt.title('Permutation Test - 50 Permutations',size=16)

#calculate and plot ROC curves for each KFold
plt.figure(figsize=(9,9))
for i,(train,test) in enumerate(cv):
    clf = clf.fit(X_values[train], y_values[train])
    prob = clf.predict_proba(X_values[test])
    fpr, tpr, thres = roc_curve(y_values[test],prob[:,1])
    auc_data = auc(fpr,tpr)
    plt.plot(fpr,
             tpr,
             lw=1,
             label="ROC Curve Fold %d, Area = %0.2f" % (i, auc_data))
plt.plot([0,1],[0,1],
         lw=5,
         label="ROC Curve, Chance",
         color='gray')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])
plt.xlabel('False Positive Rate',size=12)
plt.ylabel('True Positive Rate',size=12)
plt.title('ROC Analysis', size=16)
plt.legend(loc='lower right')

#show both plots
plt.tight_layout()
plt.show()